In [1]:
import matplotlib.pyplot as plt
from seaborn import heatmap
from seaborn import pairplot
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem
import pandas as pd
import numpy as np
import copy
import os
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

#### Making Samples

In [2]:
samples_temp = []
wwr_values = np.linspace(0.1,0.9,9)
thickness_values = [0.0001,0.05,0.35]
for thickness in thickness_values:
    for wwr_ in wwr_values:
        samples_temp.append({'Window to Wall Ratio': wwr_,
                             'Insulation Thickness': thickness,})
samples = pd.DataFrame.from_dict(samples_temp)
samples              

,Window to Wall Ratio,Insulation Thickness
0,0.1,0.0001
1,0.2,0.0001
2,0.3,0.0001
3,0.4,0.0001
4,0.5,0.0001
5,0.6,0.0001
6,0.7,0.0001
7,0.8,0.0001
8,0.9,0.0001
9,0.1,0.0500


#### Loading data and pefroming Besos evaluation

In [9]:
main_dir = '../idfv9'
results = pd.core.frame.DataFrame()

for filename in os.listdir(main_dir):
    if filename.endswith('.idf'):
        params = filename.split('.')[0].split('_')
        ventilation = params[0]
        building = ef.get_building(main_dir + '/' + filename)
        insulation = FieldSelector(class_name='Material', object_name='Cement/plaster/mortar - gypsum plaster_O.OO2', field_name='Thickness')
        insulationPR = Parameter(selector=insulation, value_descriptor=CategoryParameter(options=[0.0001,0.05,0.35]),name='Insulation Thickness')

        window_to_wall = wwr(RangeParameter(0.1,0.9))

        parameters = [window_to_wall, insulationPR]

        objectives = ['Electricity:Facility', 'DistrictHeating:Facility', 'DistrictCooling:Facility','InteriorLights:Electricity']

        problem = EPProblem(parameters, objectives)
        evaluator = EvaluatorEP(problem, building, out_dir='Output/output_besos', error_mode='Silent', err_dir='Output/output_besos',
                            epw='./epw/casaccia_ENEA-hour.epw')
        
#         samples = sampling.dist_sampler(sampling.lhs,problem, num_samples=20)
        
        outputs = evaluator.df_apply(samples, keep_input=True)    
        if ventilation == 'on':
            outputs['Ventilation'] = 'on'
        else:
            outputs['Ventilation'] = 'off'

        results = results.append(outputs)


TypeError: make_idd_index() missing 1 required positional argument: 'debug'

#### Conversion to KWh

In [7]:
results['Electricity:Facility'] /= 3.6e6
results['DistrictHeating:Facility'] /= 3.6e6
results['DistrictCooling:Facility'] /= 3.6e6
results['TotalConsumption'] = results['DistrictHeating:Facility'] + results['DistrictCooling:Facility']
results['InteriorLights:Electricity'] /= 3.6e6
results

KeyError: 'Electricity:Facility'

In [8]:
results = results.reset_index()
results

,index


In [9]:
results['Electricity:Facility'] = results['Electricity:Facility'].apply(lambda x: round(x, 2))
results['DistrictHeating:Facility'] = results['DistrictHeating:Facility'].apply(lambda x: round(x, 2))
results['DistrictCooling:Facility'] = results['DistrictCooling:Facility'].apply(lambda x: round(x, 2))
results['TotalConsumption'] = results['TotalConsumption'].apply(lambda x: round(x, 2))
results['InteriorLights:Electricity'] = results['InteriorLights:Electricity'].apply(lambda x: round(x, 2))
results

KeyError: 'Electricity:Facility'

#### showing lowest and highest power consumption

In [18]:
minValue = results['TotalConsumption'].min()
minValueLoc = results['TotalConsumption'].idxmin()
minValue
print("lowest total consumption:",minValue,",Location: row",minValueLoc)

maxValue = results['TotalConsumption'].max()
maxValueLoc = results['TotalConsumption'].idxmax()

print("highest total consumption:",maxValue,",Location: row",maxValueLoc)

lowest total consumption: 24191.36 ,Location: row 45
hihjest total consumption: 50680.63 ,Location: row 8


In [19]:
writer = pd.ExcelWriter("Output/OutputBesosResults.xlsx")
results.to_excel(writer,sheet_name='Sheet1') 
writer.save()